# Basic Usage

In [2]:
import asyncio
from agents import Agent, handoff, Runner, trace

# Define your agents
billing_agent = Agent(name="Billing agent")
refund_agent = Agent(name="Refund agent")
triage_agent = Agent(name="Triage agent", handoffs=[billing_agent, handoff(refund_agent)])


In [3]:
# Conversation thread ID
thread_id = "1"

In [8]:
# no default threading in the convo!

# Interactive chat loop
async def chat_with_agent(agent):
    print("Start chatting with the agent (type 'exit' to quit):")
    with trace(workflow_name="Triage Workflow", group_id=thread_id):
        while True:
            user_input = input("You: ")
            if user_input.lower() in ["exit", "quit"]:
                break
            result = await Runner.run(agent, user_input)
            print("Agent:", result.final_output)

# Run in Jupyter
await chat_with_agent(triage_agent)


Start chatting with the agent (type 'exit' to quit):


You:  I'm Sinan


Agent: Hi Sinan! How can I help you today?


You:  what is my name?


Agent: I'm sorry, I don't have access to personal data about you. Could you please tell me your name?


You:  exit


# Adding threading

In [10]:
# Chat loop with history
async def chat_with_agent_with_thread(agent):
    print("Start chatting with the agent (type 'exit' to quit):")
    with trace(workflow_name="Triage Workflow", group_id=thread_id):
        result = None
        while True:
            user_input = input("You: ")
            if user_input.lower() in ["exit", "quit"]:
                break

            # Build input list with history
            if result:
                new_input = result.to_input_list() + [{"role": "user", "content": user_input}]
            else:
                new_input = user_input  # First turn is just the raw input
            
            result = await Runner.run(agent, new_input)
            print("Agent:", result.final_output)


# Run in Jupyter
await chat_with_agent_with_thread(triage_agent)

Start chatting with the agent (type 'exit' to quit):


You:  I'm Sinan


Agent: Hi Sinan! How can I assist you today?


You:  what is my name?


Agent: You mentioned your name is Sinan. How can I assist you further?


You:  refund


Agent: To help with a refund, I'll connect you to the refund agent. Please provide any specific details about your request, if prompted.


You:  what is my name?


Agent: You mentioned earlier that your name is Sinan.


You:  exit


# Using a tool to get the weather and more advanced handoff instructions

In [24]:
import asyncio
from agents import Agent, Runner, handoff, function_tool
from agents.extensions.handoff_prompt import prompt_with_handoff_instructions

import random

# Define the tool
@function_tool
def get_weather(city: str) -> str:
    """Get the weather for a given city."""
    print(f"[debug] get_weather called with city: {city}")
    choices = ["sunny", "cloudy", "rainy", "snowy"]
    return f"The weather in {city} is {random.choice(choices)}."

# Turkish agent
turkish_agent = Agent(
    name="Turkish",
    handoff_description="A Turkish speaking agent.",
    instructions=prompt_with_handoff_instructions(
        "You're speaking to a human, so be polite and concise. Speak in Turkish.",
    ),
    model="gpt-4o-mini",
)

# Main assistant agent
agent = Agent(
    name="Assistant",
    instructions=prompt_with_handoff_instructions(
        "You're speaking to a human, so be polite and concise. If the user speaks in Turkish, handoff to the turkish agent.",
    ),
    model="gpt-4o-mini",
    handoffs=[turkish_agent],
    tools=[get_weather],
)

# Chat loop with history
async def chat_with_agent(agent):
    print("Start chatting with the agent (type 'exit' to quit):")
    result = None
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            break

        if result:
            new_input = result.to_input_list() + [{"role": "user", "content": user_input}]
        else:
            new_input = user_input  # First turn

        result = await Runner.run(agent, new_input)
        print("Agent:", result.final_output)

# Run the chat loop in a Jupyter cell
await chat_with_agent(agent)


Start chatting with the agent (type 'exit' to quit):


You:  merhaba


Agent: Merhaba! Size nasıl yardımcı olabilirim?


You:  Istanbulda hava nasil?


[debug] get_weather called with city: Istanbul
Agent: İstanbul'da hava yağışlı. Başka bir konuda yardımcı olabilir miyim?


You:  exit
